In [2]:
# --- IMPORTS ---

import cv2
import os
import numpy as np
from PIL import Image


In [3]:
# --- generating data set for face detection ---
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    if face_classifier.empty():
        print("Error loading haarcascade_frontalface_default.xml")
        return

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    # Create output folder
    if not os.path.exists("facedata"):
        os.makedirs("facedata")

    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    if not cap.isOpened():
        print("Could not open webcam")
        return

    id = 1
    img_id = 0

    print("Starting face capture. Press Enter or wait for 200 samples.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print(" Failed to grab frame")
            break

        cropped_face = face_cropped(frame)
        if cropped_face is not None:
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = f"facedata/user.{id}.{img_id}.jpg"
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped Face", face)

        # Enter key se dataset generation end hoga ya 200 images lene ke baad
        if cv2.waitKey(1) == 13 or img_id == 200:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Samples have been collected")

if __name__ == "__main__":
    generate_dataset()


Starting face capture. Press Enter or wait for 200 samples.
Samples have been collected


In [4]:
def Train_Classifier(facedata_dir):
    path = [os.path.join(facedata_dir,f) for f in os.listdir(facedata_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        ImageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(ImageNp)
        ids.append(id)
    ids = np.array(ids)

    # --- Classifier to be trained ---
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")


Train_Classifier("facedata")


--- Face detection wala box and returning student's name below this ---

In [5]:
def DrawBoundary(img, classifier, scaleFactor, minNeighbor, color, text, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbor)

    coords = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        id, pred = clf.predict(gray_image[y:y + h, x:x + w])
        confidence = int(100 * (1 - pred / 300))

        # --- id use karni hai to fetch student details such as name, roll number, etc

        if confidence > 75:
            cv2.putText(img, str(id), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, "Unknown", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)

        coords = [x, y, w, h]

    return coords

def recognize(img, clf, face_cascade):
    coords = DrawBoundary(img, face_cascade, 1.2, 10, (255, 255, 255), "Face", clf)  # pass color as tuple
    return img

face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
if face_cascade.empty():
    print("Error loading haarcascade_frontalface_default.xml")
    exit()

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret, img = video_capture.read()
    if not ret:
        print("Failed to capture frame")
        break

    img = recognize(img, clf, face_cascade)
    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) == 13:  # Enter key to exit
        break

video_capture.release()
cv2.destroyAllWindows()
